<a href="https://colab.research.google.com/github/bhagya2002/2Chat/blob/master/449_lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install scikit-fuzzy library
!pip install scikit-fuzzy

import skfuzzy as fuzz
from skfuzzy.control import ControlSystem, ControlSystemSimulation, Antecedent, Consequent, Rule
import numpy as np

# Define input variables and fuzzy sets for Food Quality
food_temperature = np.arange(0, 11, 1)
food_flavor = np.arange(0, 11, 1)
portion_size = np.arange(0, 11, 1)

food_temperature_antecedent = Antecedent(food_temperature, 'Food Temperature')
food_flavor_antecedent = Antecedent(food_flavor, 'Food Flavor')
portion_size_antecedent = Antecedent(portion_size, 'Portion Size')

# Define input variables and fuzzy sets for Service Quality
attentiveness = np.arange(0, 11, 1)
friendliness = np.arange(0, 11, 1)
speed = np.arange(0, 11, 1)

attentiveness_antecedent = Antecedent(attentiveness, 'Attentiveness')
friendliness_antecedent = Antecedent(friendliness, 'Friendliness')
speed_antecedent = Antecedent(speed, 'Speed of Service')

# Define the universe of discourse for tip percentage (output variable)
tip_percentage = np.arange(0, 26, 1)
tip_consequent = Consequent(tip_percentage, 'Tip Percentage')

food_quality_output = Consequent(np.arange(0, 11, 1), 'Food Quality')
service_quality_output = Consequent(np.arange(0, 11, 1), 'Service Quality')

# Fuzzy set definitions
food_temperature_antecedent.automf(3)
food_flavor_antecedent.automf(3)
portion_size_antecedent.automf(3)

attentiveness_antecedent.automf(3)
friendliness_antecedent.automf(3)
speed_antecedent.automf(3)

food_quality_output.automf(3)
service_quality_output.automf(3)
tip_consequent.automf(3)

# Define fuzzy rules for Food Quality and Service Quality
rules_food_quality = [
    Rule(food_temperature_antecedent['poor'] | food_flavor_antecedent['poor'] | portion_size_antecedent['poor'], food_quality_output['poor']),
    Rule(food_temperature_antecedent['average'] & food_flavor_antecedent['average'] & portion_size_antecedent['average'], food_quality_output['average']),
    Rule(food_temperature_antecedent['good'] & food_flavor_antecedent['good'] & portion_size_antecedent['good'], food_quality_output['good'])
]

rules_service_quality = [
    Rule(attentiveness_antecedent['poor'] | friendliness_antecedent['poor'] | speed_antecedent['poor'], service_quality_output['poor']),
    Rule(attentiveness_antecedent['average'] & friendliness_antecedent['average'] & speed_antecedent['average'], service_quality_output['average']),
    Rule(attentiveness_antecedent['good'] & friendliness_antecedent['good'] & speed_antecedent['good'], service_quality_output['good'])
]

rules_tipper = [
    Rule(food_quality_output['poor'] | service_quality_output['poor'], tip_consequent['poor']),
    Rule(food_quality_output['average'] & service_quality_output['average'], tip_consequent['average']),
    Rule(food_quality_output['good'] & service_quality_output['good'], tip_consequent['good'])
]

# Create control objects for Food Quality and Service Quality
food_quality_ctrl = ControlSystem(rules=rules_food_quality)
service_quality_ctrl = ControlSystem(rules=rules_service_quality)

# Create control system simulations for Food Quality and Service Quality
food_quality_sim = ControlSystemSimulation(food_quality_ctrl)
service_quality_sim = ControlSystemSimulation(service_quality_ctrl)

# Combine all the rules into one set
all_rules = rules_food_quality + rules_service_quality + rules_tipper

# Create the control system with all the rules
tipper_ctrl = ControlSystem(rules=all_rules)
tipper_sim = ControlSystemSimulation(tipper_ctrl)

while True:
    try:
        # Take user input for the six measures
        food_temp = float(input("Enter food temperature (0-10): "))
        food_flav = float(input("Enter food flavor (0-10): "))
        portion_s = float(input("Enter portion size (0-10): "))
        attent = float(input("Enter attentiveness (0-10): "))
        friend = float(input("Enter friendliness (0-10): "))
        service_speed = float(input("Enter service speed (0-10): "))

        # Validate inputs
        for val in [food_temp, food_flav, portion_s, attent, friend, service_speed]:
            if not 0 <= val <= 10:
                raise ValueError("Please enter valid inputs between 0 and 10.")
    except ValueError as e:
        print(e)
        continue  # Continue will jump to the next iteration of the loop

    try:
        # Set input values for Food Quality
        food_quality_sim.input['Food Temperature'] = food_temp
        food_quality_sim.input['Food Flavor'] = food_flav
        food_quality_sim.input['Portion Size'] = portion_s

        # Compute Food Quality
        food_quality_sim.compute()

        # Set input values for Service Quality
        service_quality_sim.input['Attentiveness'] = attent
        service_quality_sim.input['Friendliness'] = friend
        service_quality_sim.input['Speed of Service'] = service_speed

        # Compute Service Quality
        service_quality_sim.compute()

        # Set input values directly into the tipper_sim for all the primary parameters
        tipper_sim.input['Food Temperature'] = food_temp
        tipper_sim.input['Food Flavor'] = food_flav
        tipper_sim.input['Portion Size'] = portion_s
        tipper_sim.input['Attentiveness'] = attent
        tipper_sim.input['Friendliness'] = friend
        tipper_sim.input['Speed of Service'] = service_speed

        # Compute the tip
        tipper_sim.compute()

        # Display the results
        print(f"\nFood Quality Score: {food_quality_sim.output['Food Quality']:.2f}/10")
        print(f"Service Quality Score: {service_quality_sim.output['Service Quality']:.2f}/10")
        print(f"Recommended Tip Percentage: {tipper_sim.output['Tip Percentage']:.2f}%")
    except Exception as e:
        print(f"An error occurred during fuzzy computations: {e}")

    # Ask the user if they want to enter more values
    another_meal = input("Enter another set of measures? (y/n): ")
    if another_meal.lower() != 'y':
        break



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894079 sha256=f35c687d0486d97f940d9cb49d74c2949db05196745ee422e239d2e859fb0415
  Stored in directory: /root/.cache/pip/wheels/4f/86/1b/dfd97134a2c8313e519bcebd95d3fedc7be7944db022094bc8
Successfully built scikit-fuzzy
Enter food temperature (0-10): 2
Enter food flavor (0-10): 6
Enter portion size (0-10): 10
Enter attentiveness (0-10): 1
Enter friendliness (0-10): 4
Enter service speed (0-10): 9

Food Quality Score: 1.86/10
Service Quality Score: 3.27/10
Recommended Tip Percentage: 4.32%
Enter another set of measures? (y/n): y
Enter food temperature (0-10): 0
Enter food flavor (0-10): 0
Enter portion size (0-10): 0
Enter attentiveness (0-10): 0
Enter friendliness (0-10): 0
Enter service speed (0-10): 0

Food Quality Score: 1.67/10
Service Quality Score: 1.67/10
Recommended Tip Perce